# Beat Generation - DevFest Nantes 2021 - IArt

 
<br>  
Brique n°3 de la conférence **IArt ou comment apprendre à une machine à tagger et à composer**
 
<br>  

<img src="images/image1.jpg" alt="Beat Generation" />

### Objectifs
 
<br>  
- Introduction à la génération d'un signal audio
<br>  
<br>  
- Créer un beat pour notre tube 🎸🤘🎼🎵♬

### Etapes

<br>  
1. On va lire l'ensemble des fichiers Midi à disposition
<br>  
    - Dans un premier temps, on va analyser les fichiers pour définir un interval de temps fix entre deux notes.
    - Ensuite, on va lire tous les fichiers et concaténer les notes dans une seule liste (avec un écart de temps entre deux fichiers).  
<br>  
2. Création d'une séquence de "classes" à prédire
<br>  
    - On va créer une "classe" par couple note/vélocité
    - On va ensuite "compléter" les intervals de temps par des "blancs" pour avoir une note par interval de temps.  
<br>  
3. Gestion des "blancs"
<br>  
    - Après quelques expériences, on s'est rendu compte qu'il fallait mieux regrouper les "blancs" par groupes  
        - En effet, on peut avoir plus de 20 "blancs" à suivre, ce qui va complètement fausser notre modèle.  
        - Dans les fait, on peut quand même obtenir des bons résultats, mais ça nécessite un apprentissage très long sur un modèle complexe.  

    - On analyse donc la séquence de notes (classes) et on va regrouper les "blancs" dans des nouvelles "méta"-classes de "blancs"  
<br>  
4. Modelisation
<br>  
    - On définit la cible de notre modèle -> prédire la note suivante d'une séquence.
    - On créé un modèle récurrent qui prend en entrée une séquence de note.  
<br>  
5. Entrainement
<br>  
    - On entraine le modèle sur l'intégralité de nos données.
        - Pas obligé d'avoir un jeu de validation, on veut volontairement overfitter le modèle pour apprendre au mieux le "genre" des beats en entrée.  
<br>  
6. Prédictions
<br>  
    - Enfin, on va vouloir utiliser notre modèle pour créer un nouveau son !  
    - Idée :
      - Fournir une séquence en entrée (aléatoire, une seule note, ou encore une séquence choisie manuellement, ...)
      - Prédire la prochaine note
      - Recréer une séquence avec la nouvelle note (moins la première note de la séquence précédente)
      - Continuer jusqu'à avoir un certain nombre de notes  
    - La fonction de génération inclut une sélection "aléatoire" selon la distribution des prédictions obtenues par le modèle, pour apporter un peu de variété.  

---

### Imports

In [1]:
# Divers
import os
import sys
import copy
import glob
import time
import random
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display, HTML, Javascript, clear_output

# Musique
import mido
from mido import Message, MidiFile, MidiTrack, second2tick, MetaMessage

# TensorFlow
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (ELU, LSTM, BatchNormalization, Bidirectional, Embedding, Softmax,
                                     Dense, Dropout, Input, LeakyReLU, ReLU, GRU, SpatialDropout1D)
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Center figures
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

---

### Fonctions utilitaires

<br>  
On commence par définir certaines fonctions utilitaires :

- **hide_toggle** : permet de "masquer" certaines cellules de ce notebook, ce qui permet d'apporter un peu plus de clarté ;)   

- **load_midi_file** : Fonction pour lire un fichier midi   

- **play_midi_messages** : Fonction pour lire une suite de notes Midi

In [3]:
def hide_toggle(for_next=False, text_display='Toggle show/hide'):
    '''Function to hide a notebook cell'''
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    target_cell = this_cell  # target cell to control with toggle

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = f"""
        <script>
            function {js_f_name}() {{
                {target_cell}.find('div.input').toggle();
            }}

        </script>

        <a href="javascript:{js_f_name}()" id="{js_f_name}">{text_display}</a>
    """
    
    js = f'''
            var output_area = this;
            var cell_element = output_area.element.parents('.cell');
            var cell_idx = Jupyter.notebook.get_cell_elements().index(cell_element);
            var current_cell = Jupyter.notebook.get_cell(cell_idx);
            $(current_cell.element[0]).find('div.input').toggle();
            Jupyter.notebook.select(cell_idx +  1);
            Jupyter.notebook.focus_cell();
         '''

    display(HTML(html))
    display(Javascript(js))

hide_toggle(text_display='Toggle show/hide --- fonction hide_toggle')

<IPython.core.display.Javascript object>

In [4]:
def load_midi_file(file_path: str, min_time_gap: float = 0.):
    '''Fonction pour lire un fichier midi
    
    Args:
        file_path (str): chemin du fichier à lire
        min_time_gap (float): écart minimum entre deux notes
            Permet de ne pas avoir deux notes en même temps à prédire pour notre algorithme
            Default 0. -> pas de décalage
            Généralement on veut avoir min_time_gap assez faible pour ne pas avoir de différence sur le rendu
    Returns:
        list : liste des notes du fichier Midi
    '''
    messages = []
    for i, msg in enumerate(MidiFile(file_path)):
        if not msg.is_meta:
            # Si deux notes en même temps, on décale la deuxième pour la mettre juste après la première
            # i.e. temps = temps minimum entre deux notes
            if msg.time < min_time_gap:
                msg.time = min_time_gap
            messages.append(msg)
    return messages


hide_toggle(text_display='Toggle show/hide --- fonction load_midi_file')

<IPython.core.display.Javascript object>

In [5]:
def play_midi_messages(messages):
    '''Fonction pour lire une suite de notes Midi
    
    Args:
        messages (list): notes à jouer
    '''
    outport = mido.open_output()
    for i, msg in enumerate(messages):
        time.sleep(msg.time)
        outport.send(msg)
    outport.close()

hide_toggle(text_display='Toggle show/hide --- fonction play_midi_messages')

<IPython.core.display.Javascript object>

In [6]:
def save_midi_messages(midi_messages: list, new_file_path: str):
    '''Fonction pour sauvegarder une liste de messages MIDI en un fichier midi
    
    De nombreux paramètres sont fixés après analyse de nos données
    
    Args:
        midi_messages (list): liste de messages à sauvegarder
        new_file_path (str): chemin du fichier à créer
    '''
    # Init midi file
    mid = MidiFile()
    
    # On ajoute des infos spécifiques à nos données    
    mid.type = 1 # c.f. `MidiFile("original_beats/078 Hip Hop Beat 1A.mid").type`
    # c.f. `MidiFile("original_beats/078 Hip Hop Beat 1A.mid").tracks[0]`
    mid.tracks.append(
        MidiTrack(
            [
                MetaMessage('track_name', name='AI generated song', time=0),
                MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
                MetaMessage('key_signature', key='C', time=0),
                MetaMessage('set_tempo', tempo=769231, time=0),
                MetaMessage('end_of_track', time=0),
            ])
    )

    # Notre track
    track = MidiTrack()
    for msg in midi_messages:
        # Time en ticks pour sauvegarde en midi
        # ticks_per_beat: MidiFile("original_beats/078 Hip Hop Beat 1A.mid").ticks_per_beat
        # tempo: MidiFile("original_beats/078 Hip Hop Beat 1A.mid").tracks[0][3].tempo
        # On va créer un nouveau message, car on ne veut pas modifier l'original
        msg_tick_time = round(second2tick(msg.time, 240, 769231)) * 2 # x 2 donne le bon timing. TODO : se renseigner sur la librairie mido
        final_msg = Message('note_on', channel=9, note=msg.note, velocity=msg.velocity, time=msg_tick_time)
        track.append(final_msg)
    mid.tracks.append(track)

    # Save
    mid.save(new_file_path)

hide_toggle(text_display='Toggle show/hide --- fonction save_midi_messages')

<IPython.core.display.Javascript object>

---

### 1. Lecture des fichiers Midi

<br>

- On va créer une suite de notes qui sera la concaténation de tous les fichiers à disposition  
- On ajoute un petit écart de temps entre deux fichiers  
- Au final on voudra une suite de notes avec un interval de temps fix entre deux notes  
  - On définit la valeur de cet interval de temps à la suite d'une analyse sur les fichiers à disposition
  - Dans la suite, on va remplir les trous avec des notes "blancs"

In [7]:
# On analyse les données
list_times = []
for file in glob.glob("original_beats/*.mid"): # On parcours l'ensemble des fichiers midi à disposition
    file_midi_messages = load_midi_file(file)
    for message in file_midi_messages:
        if message.time != 0:
            list_times.append(message.time)
min_interval = min(list_times)
print(f"L'écart minimum entre deux notes est égal à {round(min_interval, 5)}")

L'écart minimum entre deux notes est égal à 0.00321


In [8]:
# On définit certains paramètres
time_split_files = 0.1 # On ajoute un temps entre deux fichiers (pour ne pas les enchainer trop vite !)
interval_time = min_interval * 4 # On augmente l'écart minimum entre deux notes pour diminuer le nombre de "blancs"

In [9]:
# Chargement de l'ensemble des notes
midi_messages = []
for file in glob.glob("original_beats/*.mid"):
    file_midi_messages = load_midi_file(file)
    for i, msg in enumerate(file_midi_messages):
        # On ajoute un temps entre deux fichiers
        if i == 0:
            msg.time = 0.1
        # Si deux notes se suivent à moins de interval_time, on décale la deuxième pour la mettre juste après la première
        elif msg.time < interval_time:
            msg.time = interval_time
        midi_messages.append(msg)
midi_messages[0].time = 0 # On set le premier temps à 0

In [10]:
# On joue les première notes pour vérifier la cohérence du signal
play_midi_messages(midi_messages[:80])

---

### 2. Création d'une séquence de "classes" à prédire

<br>

- On va créer une "classe" par couple note/vélocité
- On va ensuite "compléter" les intervals de temps par des "blancs" pour avoir une note par interval de temps.  


In [11]:
# Analyse des notes de notre dataset
print(f"On a {len(set([msg.note for msg in midi_messages]))} notes différentes dans notre jeu de données")
print(f"On a {len(set([(msg.note, msg.velocity) for msg in midi_messages]))} couples notes/vélocité différents dans notre jeu de données")

On a 7 notes différentes dans notre jeu de données
On a 101 couples notes/vélocité différents dans notre jeu de données


Si on avait plus de couples différents, on pourrait seuiller les vélocités pour réduire ce nombre.

In [12]:
# On définit une classe par couple note/vélocité
available_couples = sorted(list(set([f'{msg.note}_{msg.velocity}' for msg in midi_messages])))
# On créé un "tokenizer" : note -> token
notes_to_tokens = {note: i for i, note in enumerate(available_couples)}
# On affiche 5 éléments pour exemple
print(random.sample(available_couples, 5))

['36_116', '36_110', '37_100', '42_64', '38_103']


In [13]:
# On rajoute une note "blanc"
notes_to_tokens['blanc'] = len(notes_to_tokens)

In [14]:
# On définit le dictionnaire inverse
classes_to_notes = {value: key for key, value in notes_to_tokens.items()}

In [15]:
# On créé une séquence de notes (classes) en complétant les intervals par des "blancs"
notes_seq = []
for msg in midi_messages:
    cl = f'{msg.note}_{msg.velocity}'
    nb_blanc_steps = round(msg.time/interval_time) - 1
    if nb_blanc_steps > 0:
        notes_seq += ['blanc'] * nb_blanc_steps
    notes_seq.append(cl)

# On affiche le début
print(notes_seq[:10])

['42_69', '36_114', 'blanc', 'blanc', 'blanc', 'blanc', 'blanc', 'blanc', 'blanc', '42_0']


---

### 3. Gestion des "blancs"

<br>

- Après quelques expériences, on s'est rendu compte qu'il fallait mieux regrouper les "blancs" par groupes  
    - En effet, on peut avoir plus de 20 "blancs" à suivre, ce qui va risque de fausser notre modèle.  
    - Dans les fait, on peut quand même obtenir des bons résultats, mais ça nécessite un apprentissage très long sur un modèle complexe.  
- On analyse donc la séquence de notes (classes) et on va regrouper les "blancs" dans des nouvelles "méta"-classes de "blancs" 
<br>   
<br>   

On définit quelques fonctions pour clarifier le notebook :

- **cnt_blancs** : Fonction pour compter les "blancs" dans une séquence  
- **regroup_blancs** : Fonction pour regrouper les "blancs" dans des "méta"-classes de blancs


In [16]:
def cnt_blancs(seq: list):
    '''Fonction pour compter les "blancs" dans une séquence
    
    Args:
        seq (list): séquence à analyser
    '''
    # On commence par regarder les classes de 'blancs' présentes dans la séquence
    list_classes_blancs = list(set([note for note in seq if note.startswith('blanc')]))
    
    # Analyse des occurrences
    for cl_blanc in list_classes_blancs:
        print(f"On a {round(seq.count(cl_blanc) / len(seq) * 100, 2)} % de note '{cl_blanc}' dans notre séquence de notes.")
    print(f"On a {round(len([_ for _ in seq if not _.startswith('blanc')]) / len(seq) * 100, 2)} % de 'vraies' notes dans notre séquence de notes.")
        
    # Analyse des suites
    for cl_blanc in list_classes_blancs:
        print(f"\nAnalyse de la classe '{cl_blanc}':\n")
        # On récupère le compte des suite
        nb_blanc = 0
        blancs_cnt = {}
        for note in seq:
            if note == cl_blanc:
                nb_blanc += 1
            elif nb_blanc != 0:
                blancs_cnt[nb_blanc] = blancs_cnt.get(nb_blanc, 0) + 1
                nb_blanc = 0
        # Affichage
        for key in sorted(list(blancs_cnt.keys())):
            print(f"    Il y a {blancs_cnt[key]} suites de {key} '{cl_blanc}'")

hide_toggle(text_display='Toggle show/hide --- fonction cnt_blancs')

<IPython.core.display.Javascript object>

In [17]:
def regroup_blancs(notes_seq: list, groupes_dict: dict):
    '''Fonction pour regrouper les "blancs" dans des "méta"-classes de blancs
    
    Args:
        seq (list): séquence à analyser
        groupes_dict (dict): dictionnaire avec les regroupements à faire
            clé : nb d'occurrences à suivre
            valeur : nouvelle classe à associer
    Returns:
        list: nouvelle séquence mise à jour
    '''
    min_notes = min(groupes_dict.keys()) # nombre minimal de blancs pour regroupement, d'après nos choix
    
    # On va parcourir la séquence jusqu'à trouver une suite de blancs éligible
    # A chaque suite trouvée, on met à jour la séquence de notes
    # Il faut donc recommencer le parcours avec cette nouvelle séquence (d'où le while)

    work_to_do = True # Tant qu'il y a des suites de blancs éligibles
    while work_to_do:
        started = False # Indique si on a démarré l'étude d'une suite de 'blancs'
        breaked = False # Indique si la boucle for à été break
                        # i.e. on a trouvé une séquence de blanc, on reparcours le séquence depuis le début
        cnt = 0 # Nb de blancs identifiés
        start_index = 0 # Index du début de la suite de blancs à l'étude

        # On parcours l'intégralité de la séquence de notes
        for i, note in enumerate(notes_seq):

            # Init cnt
            if note == 'blanc' and started == False:
                start_index = i
                started = True
                cnt = 1

            # Another blanc
            elif started == True and note == 'blanc':
                cnt += 1

            # Not a blanc
            elif started == True and note != 'blanc':
                # Pas assez de notes pour regrouper
                if cnt < min_notes:
                    started=False
                    cnt = 0
                # Assez de notes, on regroupe
                else:
                    end_index = i
                    notes_seq = notes_seq[:start_index] + [groupes_dict[cnt]] + notes_seq[end_index:]
                    # On indique qu'il faut continuer
                    breaked = True
                    # On break la boucle
                    break        

        if not breaked:
            # Si on n'a pas trouvé de suite éligible dans l'intégralité de la séquence, on quitte la boucle
            work_to_do = False
    
    # Return nouvelle séquence
    return notes_seq

hide_toggle(text_display='Toggle show/hide --- fonction regroup_blancs')

<IPython.core.display.Javascript object>

In [18]:
# Analyse des 'blancs'
cnt_blancs(notes_seq)

On a 88.23 % de note 'blanc' dans notre séquence de notes.
On a 11.77 % de 'vraies' notes dans notre séquence de notes.

Analyse de la classe 'blanc':

    Il y a 3 suites de 1 'blanc'
    Il y a 36 suites de 5 'blanc'
    Il y a 264 suites de 6 'blanc'
    Il y a 479 suites de 7 'blanc'
    Il y a 9 suites de 8 'blanc'
    Il y a 6 suites de 20 'blanc'
    Il y a 357 suites de 21 'blanc'
    Il y a 45 suites de 22 'blanc'
    Il y a 24 suites de 23 'blanc'


On a trop de 'blancs', on décide donc de faire des regroupements
- On regroupe 5 & 6 ensemble
- On regroupe 7 & 8 ensemble
- On regroupe 20, 21, 22 & 23 ensemble

In [19]:
# On ajout les nouvelles classes de "blancs"
new_notes = ['blanc_6', 'blanc_7', 'blanc_21']
for note in new_notes:
    notes_to_tokens[note] = len(notes_to_tokens)
    classes_to_notes[len(classes_to_notes)] = note

In [20]:
# On créé un dictionnaire de regroupement de 'blancs'
cnt_to_notes_dict = {
    5: 'blanc_6', # On regroupe 5 & 6 ensemble
    6: 'blanc_6', # On regroupe 5 & 6 ensemble
    7: 'blanc_7', # On regroupe 7 & 8 ensemble
    8: 'blanc_7', # On regroupe 7 & 8 ensemble
    20: 'blanc_21', # On regroupe 20, 21, 22 & 23 ensemble
    21: 'blanc_21', # On regroupe 20, 21, 22 & 23 ensemble
    22: 'blanc_21', # On regroupe 20, 21, 22 & 23 ensemble
    23: 'blanc_21', # On regroupe 20, 21, 22 & 23 ensemble
}

In [21]:
# On transforme notre séquence pour y intégrer les regroupements de blancs
final_notes_seq = regroup_blancs(notes_seq, cnt_to_notes_dict)

In [22]:
# On analyse notre nouvelle séquence
cnt_blancs(final_notes_seq)

On a 15.56 % de note 'blanc_7' dans notre séquence de notes.
On a 13.77 % de note 'blanc_21' dans notre séquence de notes.
On a 0.1 % de note 'blanc' dans notre séquence de notes.
On a 9.56 % de note 'blanc_6' dans notre séquence de notes.
On a 61.01 % de 'vraies' notes dans notre séquence de notes.

Analyse de la classe 'blanc_7':

    Il y a 488 suites de 1 'blanc_7'

Analyse de la classe 'blanc_21':

    Il y a 432 suites de 1 'blanc_21'

Analyse de la classe 'blanc':

    Il y a 3 suites de 1 'blanc'

Analyse de la classe 'blanc_6':

    Il y a 300 suites de 1 'blanc_6'


Cette fois-ci on a plus de 60% de 'vraies' notes, c'est mieux !

---

### 4. Modelisation

<br>

- On définit la cible de notre modèle -> prédire la note suivante d'une séquence.
- On créé un modèle récurrent qui prend en entrée une séquence de note.

In [23]:
# Paramètres
sequence_length = 200

In [24]:
# On crée les entrées/sorties de notre modèle
# Il s'agit de l'ensemble des séquences de longueur 'sequence_length', et de la prochaine note à prédire
network_input = []
network_output = []
for i in range(0, len(final_notes_seq) - sequence_length, 1):
    sequence_in = final_notes_seq[i:i + sequence_length]
    sequence_out = final_notes_seq[i + sequence_length]
    network_input.append([notes_to_tokens[note] for note in sequence_in])
    network_output.append(notes_to_tokens[sequence_out])

# On reshape nos données pour être compatible avec un réseau de neurones récurrent
network_input = np.reshape(network_input, (len(network_input), sequence_length))
network_output = to_categorical(network_output)

In [25]:
# Définition de notre modèle récurrent
num_classes = network_output.shape[1]
embedding_size = 10 # Paramètrable

# Set model
model_in = Input(shape=(sequence_length,))
# Embedding : On ajoute une classe pour permettre des notes OOV
#             embeddings_initializer = zeros ? (pour gestion OOV)
x = Embedding(num_classes + 1, embedding_size, trainable=True)(model_in)
# x = LSTM(512, return_sequences=True)(x)  --> On peut rajouter une couche LSTM, mais long et pas très utile
x = LSTM(256, return_sequences=False)(x)
x = BatchNormalization(momentum=0.9)(x)
x = Dropout(0.1)(x)
x = Dense(256, activation=None, kernel_initializer="he_uniform")(x)
x = BatchNormalization(momentum=0.9)(x)
x = ELU(alpha=1.0)(x)
x = Dropout(0.1)(x)

# Last layer - On fait de la classification mono-label / multi-classes : activation softmax
activation = 'softmax'
out = Dense(num_classes, activation=activation, kernel_initializer='glorot_uniform')(x)

# Compile model
model = Model(inputs=model_in, outputs=[out])
lr = 0.01
optimizer = Adam(lr=lr)
# Loss / Metrics - On fait de la classification mono-label / multi-classes : categorical accuracy
metrics = ['categorical_accuracy']
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metrics)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 10)           1060      
_________________________________________________________________
lstm (LSTM)                  (None, 256)               273408    
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)              

---

### 5. Entrainement

<br>

- On entraine le modèle sur l'intégralité de nos données.
    - Pas obligé d'avoir un jeu de validation, on veut volontairement overfitter le modèle pour apprendre au mieux le "genre" des beats en entrée.

In [26]:
# On va créer un dossier où sauvegarder tous nos modèles & predictions
subfolder_name = datetime.now().strftime(f"experimentation_%Y_%m_%d-%H_%M_%S")
os.makedirs(subfolder_name)
subfolder_path = os.path.abspath(subfolder_name)
print(f"L'ensemble des données seront sauvegardées dans le répertoire {subfolder_path}")

L'ensemble des données seront sauvegardées dans le répertoire C:\Users\Alexandre\Dev\Valeuriad\devfest-2021\03_beat-generation\experimentation_2021_10_17-17_02_02


In [27]:
# Apprentissage
nb_epochs = 200 # Paramétrable -> plus cette valeure augmente, plus le modèle va apprendre par coeur
batch_size = 128 # Paramétrable

# Attention, on sauvegarde le modèle à chaque amélioration !
filepath = os.path.join(subfolder_path, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5")
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=nb_epochs, batch_size=batch_size, callbacks=callbacks_list)

Epoch 1/200
23/23 [==============================] - ETA: 0s - loss: 2.5705 - categorical_accuracy: 0.43 - 1s 30ms/step - loss: 2.5705 - categorical_accuracy: 0.4317
Epoch 2/200
23/23 [==============================] - 1s 29ms/step - loss: 1.5345 - categorical_accuracy: 0.5918
Epoch 3/200
23/23 [==============================] - 1s 29ms/step - loss: 1.2435 - categorical_accuracy: 0.6544
Epoch 4/200
23/23 [==============================] - 1s 29ms/step - loss: 1.0885 - categorical_accuracy: 0.6816
Epoch 5/200
23/23 [==============================] - 1s 29ms/step - loss: 0.9572 - categorical_accuracy: 0.7143
Epoch 6/200
23/23 [==============================] - ETA: 0s - loss: 0.8427 - categorical_accuracy: 0.73 - 1s 28ms/step - loss: 0.8427 - categorical_accuracy: 0.7365
Epoch 7/200
23/23 [==============================] - 1s 29ms/step - loss: 0.7816 - categorical_accuracy: 0.7579 0s - loss: 0.7775 - categorica
Epoch 8/200
23/23 [==============================] - 1s 29ms/step - loss: 0.7

Epoch 71/200
23/23 [==============================] - 1s 27ms/step - loss: 0.1106 - categorical_accuracy: 0.9653 0s - loss: 0.0707 - categorica
Epoch 72/200
23/23 [==============================] - 1s 27ms/step - loss: 0.1127 - categorical_accuracy: 0.9646 0s - loss: 0.1112 - categorica
Epoch 73/200
23/23 [==============================] - 1s 27ms/step - loss: 0.1034 - categorical_accuracy: 0.9683
Epoch 74/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0906 - categorical_accuracy: 0.9748
Epoch 75/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0872 - categorical_accuracy: 0.9751
Epoch 76/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0682 - categorical_accuracy: 0.9782 0s - loss: 0.0706 - categori
Epoch 77/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0696 - categorical_accuracy: 0.9792 0s - loss: 0.0605 - categori
Epoch 78/200
23/23 [==============================] - 1s 29ms/step - loss: 0.0448 - categ

23/23 [==============================] - 1s 27ms/step - loss: 0.0846 - categorical_accuracy: 0.9724
Epoch 138/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0850 - categorical_accuracy: 0.9748
Epoch 139/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0699 - categorical_accuracy: 0.9782
Epoch 140/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0701 - categorical_accuracy: 0.9799
Epoch 141/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0986 - categorical_accuracy: 0.9697
Epoch 142/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0783 - categorical_accuracy: 0.9813
Epoch 143/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0902 - categorical_accuracy: 0.9768
Epoch 144/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0771 - categorical_accuracy: 0.9809
Epoch 145/200
23/23 [==============================] - 1s 27ms/step - loss: 0.0502 - categorical_accur

In [28]:
# On liste les models, et le n° de l'epoch
models_list = [(f, int(f.split('-')[2])) for f in os.listdir(subfolder_path) if f.startswith('weights-improvement')]
# On sort par epoch
models_list = sorted(models_list, key=lambda x: x[1])

In [29]:
# Optionnel : on va garder les X meilleurs modèles pour faire de la place
nb_to_keep = 5
if len(models_list) > nb_to_keep:
    for f in models_list[:-nb_to_keep]:
        file_path = os.path.join(subfolder_path, f[0])
        os.remove(file_path)
models_list = models_list[-5:]

In [30]:
# On recharge le meilleur modèle
# ou le modèle de votre choix : les résultats peuvent être plus cohérents sur des modèles même moins bons
model.load_weights(os.path.join(subfolder_path, models_list[-1][0]))

In [31]:
# On sauvegarde le modèle en entier
# Pourquoi ? Car si on change la structure de notre modèle, on ne peut pas recharger les poids
model.save(os.path.join(subfolder_path, 'model.hdf5'))
# Pour reload : model = load_model(path_to_hdf5)

---

### 6. Prédiction

<br>

- Enfin, on va vouloir utiliser notre modèle pour créer un nouveau son !  
- Idée :  
  - Fournir une séquence en entrée (aléatoire, une seule note, ou encore une séquence choisie manuellement, ...)  
  - Prédire la prochaine note  
  - Recréer une séquence avec la nouvelle note (moins la première note de la séquence précédente)  
  - Continuer jusqu'à avoir un certain nombre de notes  
- La fonction de génération inclut une sélection "aléatoire" selon la distribution des prédictions obtenues par le modèle, pour apporter un peu de variété.
<br>   
<br>   

On définit quelques fonctions pour clarifier le notebook :

- **generate_new_pattern** : Fonction pour générer un nouveau son
- **new_song_as_midi_messages** : Function pour transformer une suite de notes en messages MIDI

In [32]:
def generate_new_pattern(model, notes_to_tokens: dict, nb_notes: int = 150, entry_note: str = None, entry_pattern: list = None,
                         use_random_distribution: bool = False, temperature: float = 1.0):
    '''Fonction pour générer un nouveau son
    On prend notre modèle entrainé, et on génère un son à partir d'un pattern en entrée.
    Soit : 
        - Un pattern aléatoire. On génère une séquence aléatoire, mais en respectant la répartition des notes.
        - Une note seule en entrée
        - Un pattern 'custom' en entrée

    Args:
        model: modèle à utiliser
        notes_to_tokens (dict): tokenizer en entrée du modèle
        nb_notes (int): nombre de notes à générer
        entry_note (str): note seule en entrée
            Si précisée, on va commencer avec un pattern avec des fausses notes, puis la note souhaitée
        entry_pattern (list): pattern à utiliser en entrée
            Si inférieur à la taille de séquence du modèle, on ajoute des fausses notes au début
            Si supérieur, on garde que les notes de fin
            
            Si entry_note & entry_pattern de précisé, on retourne une erreur.
            Si aucun des deux, on va créer un pattern aléatoire
        use_random_distribution (bool): si la prochaine note est choisie aléatoirement selon la distribution des prédictions
            ou si on prend systématiquement la plus probable
        temperature (float): paramètre qui influence sur le choix "aléatoire" du prochain caractère
            < 1.0 -> on va favoriser le caractère le plus probable
            > 1.0 -> on va donner plus de "chances" aux autres caractères
    Returns:
        prediction_output (list): notre chanson !
    '''
    # Manage errors
    if (entry_note is not None) and (entry_pattern is not None):
        raise AttributeError("On ne peut pas avoir entry_note & entry_pattern de set en même temps")

    ###
    ### Create entry pattern
    ###

    sequence_length = model.input.shape[-1]
    token_oov = len(notes_to_tokens) # Out of Vocabulary token (i.e. 'fausse' note)
    tokens_to_notes = {v: k for k, v in notes_to_tokens.items()}
    
    # Entry note
    if entry_note is not None:
        print(f"Création d'une séquence avec une seule note : {entry_note}")
        tokenized_pattern = [token_oov for i in range(sequence_length-1)]
        tokenized_pattern.append(notes_to_tokens[entry_note])
        # Prepare model format
        tokenized_pattern = np.reshape(tokenized_pattern, (sequence_length, 1))
        # Set initial output sequence
        prediction_output = [entry_note]
        
    # Entry pattern
    elif entry_pattern is not None:
        print("Création d'une séquence avec un pattern en entrée")
        # Gestion pattern trop grand
        if len(entry_pattern) >= sequence_length:
            entry_pattern = entry_pattern[-sequence_length:]
            tokenized_pattern = [notes_to_tokens[note] for note in entry_pattern]
        # Gestion pattern trop petit
        elif len(entry_pattern) < sequence_length:
            tokenized_pattern = [token_oov for i in range(sequence_length-len(entry_pattern))]
            for note in entry_pattern:
                tokenized_pattern.append(notes_to_tokens[note])
        # Prepare model format
        tokenized_pattern = np.reshape(tokenized_pattern, (sequence_length, 1))
        # Set initial output sequence
        prediction_output = entry_pattern

    # Random !
    else:
        print("Création d'une séquence avec un pattern aléatoire en entrée")
        pattern = [random.choice(list(notes_to_tokens.keys())) for i in range(sequence_length)]
        tokenized_pattern = [notes_to_tokens[note] for note in pattern]
        # Prepare model format
        tokenized_pattern = np.reshape(tokenized_pattern, (sequence_length, 1))
        # Set initial output sequence
        prediction_output = []

    ###
    ### Generate notes
    ###

    # Si on veut une répartition aléatoire selon la distribution, c'est plus simple de travailler sur les logits
    # En effet, permet d'utiliser le paramètre "temperature" pour contrôller l'importance de l'aléatoire
    if use_random_distribution:
        model.layers[-1].activation = None
    else:
        model.layers[-1].activation = Softmax()
    
    for i, note_index in enumerate(range(nb_notes)):
        # Display
        if i % 10 == 0 or i == nb_notes - 1:
            sys.stdout.write(f"\r{i+1}/{nb_notes}")
        
        # Predict
        prediction_input = np.reshape(tokenized_pattern, (1, len(tokenized_pattern), 1))
        prediction = model(prediction_input, training=False)

        if use_random_distribution:
            # On récupère une note "aléatoire" selon la distribution obtenue
            prediction = prediction / temperature
            predicted_ix = tf.random.categorical(prediction, num_samples=1)
            index = predicted_ix.numpy()[0][0]
        else:
            # On récupère la classe (note) avec la plus haute probabilité, et on l'ajoute à notre chanson !
            index = np.argmax(prediction)

        result = tokens_to_notes[index]
        prediction_output.append(result)

        # On met en place la sequence suivante
        tokenized_pattern = np.append(tokenized_pattern[1:], [index])
        tokenized_pattern = np.reshape(tokenized_pattern, (sequence_length, 1))
    
    # On remet l'activation à softmax
    model.layers[-1].activation = Softmax()
    
    # Return
    return prediction_output

hide_toggle(text_display='Toggle show/hide --- fonction generate_new_pattern')

<IPython.core.display.Javascript object>

In [33]:
def new_song_as_midi_messages(new_song: list, interval_time: float):
    '''Function pour transformer une suite de notes en messages MIDI
    
    Args:
        new_song (list): list de notes (classes)
        interval_time (float): interval de temps entre deux notes
    Returns:
        list: list of midi messages
    '''
    # On va recréer les messages MIDI
    # Pour chaque note, on va ajouter à la variable "time" autant de fois "interval_time"
    # qu'il y a eu de blancs depuis la dernière note.
    midi_messages = []
    nb_blanc = 0 # On compte le nombre de blancs
    for new_note in new_song:
        # Si note blanc, on rajoute au compteur de blancs à considérer
        if new_note.startswith('blanc'):
            if new_note == 'blanc':
                nb_blanc += 1
            else:
                nb_blanc += int(new_note.split('_')[-1]) # e.g. 'blanc_6' -> 6 blancs
        # Sinon, on ajoute la note à la liste de messages
        else:
            note = int(new_note.split('_')[0])
            velocity = int(new_note.split('_')[1])
            wait_time = (1 + nb_blanc) * interval_time
            channel = 9 # Spécifique à notre projet, pourrait être paramétré
            msg = mido.Message('note_on', channel=channel, note=note, velocity=velocity, time=wait_time)
            midi_messages.append(msg)
            # Reset nb_blanc
            nb_blanc = 0

    # La première note commence avec un temps à 0
    midi_messages[0].time = 0
    
    # Return
    return midi_messages

hide_toggle(text_display='Toggle show/hide --- fonction new_song_as_midi_messages')

<IPython.core.display.Javascript object>

In [34]:
# Génération d'une nouvelle chanson - sans sélection aléatoire selon la distribution des prédictions
# new_song = generate_new_pattern(model, notes_to_tokens, nb_notes=200, entry_note='36_101') # Une seule note
# new_song = generate_new_pattern(model, notes_to_tokens, nb_notes=200) # Aléatoire

# On peut aussi ajouter un peu d'aléatoire - avec sélection aléatoire selon la distribution des prédictions
# Le choix de la valeur temperature va généralement dépendre de notre modèle
# Plus le modèle est performant, plus on va souhaité augmenter cette valeur pour avoir un peu d'aléatoire dans les générations
# Sinon, on va avoir tendance à baisser cette valeur pour avoir plus de cohérence dans les beats générés
new_song = generate_new_pattern(model, notes_to_tokens, nb_notes=200, use_random_distribution = True, temperature = 1.0)

Création d'une séquence avec un pattern aléatoire en entrée
200/200

In [35]:
# Traduction au format MIDI (messages)
new_song_midi_messages = new_song_as_midi_messages(new_song, interval_time)

In [36]:
# On peut jouer notre nouveau song !
play_midi_messages(new_song_midi_messages)

In [37]:
# Save result
save_midi_messages(new_song_midi_messages, os.path.join(subfolder_path, datetime.now().strftime(f"new_song_%Y_%m_%d-%H_%M_%S.mid")))

Vous pouvez ensuite convertir votre fichier .midi en .wav avec un converter en ligne, par exemple : https://www.zamzar.com/fr/converters/audio/midi-to-wav/  
Attention, le résultat audio peut être différent en fonction du synthesizer utilisé.

Vous pouvez aussi le faire en local avec VLC : https://ourcodeworld.com/articles/read/1400/how-to-convert-a-midi-file-to-mp3-using-headless-vlc-player-with-the-cli-in-windows-10  

---

### Références :

- [How to Generate Music using a LSTM Neural Network in Keras](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5), *Sigurður Skúli*, 2017  
- [WaveNet: A Generative Model for Raw Audio](https://arxiv.org/pdf/1609.03499.pdf), *Aaron van den Oord and Sander Dieleman and Heiga Zen and Karen Simonyan and Oriol Vinyals and Alex Graves and Nal Kalchbrenner and Andrew Senior and Koray Kavukcuoglu*, 2016
